# Imports

In [ ]:
from utils.nfl_data import (get_weekly_df, get_seasonal_df, get_roster_df,
                            create_mappings, create_position_groups, create_positional_df)

# Setting Years List

The years past into this list provides us with the data in each of those seasons.

In [ ]:
year = 2023
years = [year]

# Seasonal and Weekly Data

In [ ]:
df_weekly = get_weekly_df(years=years, columns=[])

In [ ]:
df_seasonal = get_seasonal_df(years=years)

# Roster ID

For player mapping creations

In [ ]:
columns = ["player_name", "player_id", "position"]
df_roster = get_roster_df(years=years, columns=columns)

# Create Player Mappings

In [ ]:
player_map, id_map = create_mappings(df_roster=df_roster, df_seasonal=df_seasonal, df_weekly=df_weekly)

## Mapping Checks

In [ ]:
# kelce_id = id_map["Travis Kelce"]

In [ ]:
# player_map[kelce_id]["seasonal_df"]

In [ ]:
# player_map[kelce_id]["week_dfs"][-1]

# Creating Position Groups

In [ ]:
qbs, rbs, wrs, tes = create_position_groups(player_map=player_map)

In [ ]:
df_qb = create_positional_df(qbs)
df_rb = create_positional_df(rbs)
df_wr = create_positional_df(wrs)
df_te = create_positional_df(tes)

In [ ]:
df_qb.head(1)

In [ ]:
df_rb.head(1)

In [ ]:
df_wr.head(1)

In [ ]:
df_te.head(1)

# Saving Data to CSV for Graphing Tools

In [ ]:
def clean_df_for_csv(df):
    """"""
    df["name"] = df["player_id"].map(lambda x: player_map[x]['name'])
    df.drop(columns=["player_id", "Index"], inplace=True)
    
    df["total_epa"] = df["passing_epa"] + df["rushing_epa"]
    
    ignore_cols = ["name", "season", "season_type", "games", "target_share", "air_yards_share"
                  "tgt_sh", "ay_sh", "yac_sh", "ry_sh", "rtd_sh", "rfd_sh", "rtdfd_sh", "ppr_sh"]
    for col in df_qb.columns.to_list():
        if col not in ignore_cols:
            df[f"{col} per game"] = df[col] / df["games"]
            
    df = df[['name'] + [col for col in df.columns if col != "name"]]
    return df

## Quarterback Save

In [ ]:
df_to_save_qb = clean_df_for_csv(df_qb)
df_to_save_qb.head(1)

In [ ]:
df_to_save_qb.to_csv(f"FantasyData/advanced-stats/QB/{year}.csv")

## Running Back Save

In [ ]:
df_to_save_rb = clean_df_for_csv(df_rb)
df_to_save_rb.head(1)

In [ ]:
df_to_save_rb.to_csv(f"FantasyData/advanced-stats/RB/{year}.csv")

# Wide Receiver Save

In [ ]:
df_to_save_wr = clean_df_for_csv(df_wr)
df_to_save_wr.head(1)

In [ ]:
df_to_save_wr.to_csv(f"FantasyData/advanced-stats/WR/{year}.csv")

# Tight End Save

In [ ]:
df_to_save_te = clean_df_for_csv(df_te)
df_to_save_te.head(1)

In [ ]:
df_to_save_te.to_csv(f"FantasyData/advanced-stats/TE/{year}.csv")